In [3]:
from tkinter import Tk, filedialog

Tk().withdraw()
pdf_path = filedialog.askopenfilename(
    filetypes=[("PDF files", "*.pdf")]
)

print(pdf_path)


D:/NCKH/DataPhuc/144-ndcp.signed_bộ xây dựng.pdf


In [4]:
#pip install pdfplumber


In [5]:
import pdfplumber

def is_text_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
        return text is not None and len(text.strip()) > 50


In [6]:
import pdfplumber

def extract_text_pdf(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text


In [7]:
import sys
!{sys.executable} -m pip install pdfplumber pdf2image


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pdfplumber
import pdf2image
print("OK")


OK


In [9]:
from pdf2image import convert_from_path

# Chỉ định đường dẫn đến thư mục bin của Poppler
# Thay đổi đường dẫn này sau khi bạn tải và giải nén Poppler
poppler_path = r"D:\Release-25.12.0-0\poppler-25.12.0\Library\bin"  # Điều chỉnh theo đường dẫn thực tế

images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
images[0].save("page_1.png", "PNG")


In [10]:
# Cài đặt thư viện OCR
# import sys
# !{sys.executable} -m pip install pytesseract pillow


### Lưu ý: Cài đặt Tesseract OCR
Tải Tesseract OCR cho Windows từ: https://github.com/UB-Mannheim/tesseract/wiki
- Sau khi cài đặt, thêm đường dẫn vào PATH hoặc chỉ định trực tiếp trong code
- Ví dụ: `C:\Program Files\Tesseract-OCR\tesseract.exe`


In [11]:
import pytesseract
from PIL import Image
import json
import re

# Chỉ định đường dẫn Tesseract (nếu chưa có trong PATH)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def ocr_from_image(image_path):
    """Trích xuất text từ ảnh bằng OCR"""
    img = Image.open(image_path)
    # Sử dụng tiếng Việt: đảm bảo đã cài 'vie.traineddata'
    text = pytesseract.image_to_string(img, lang='vie')
    return text

def ocr_from_pdf_images(images):
    """Trích xuất text từ danh sách ảnh (đã convert từ PDF)"""
    full_text = ""
    total = len(images)
    
    for i, img in enumerate(images):
        # Chỉ hiển thị progress mỗi 5 trang để giảm output
        if i == 0 or (i+1) % 5 == 0 or (i+1) == total:
            print(f"   🔄 Đang xử lý: {i+1}/{total} trang...", end='\r')
        
        try:
            text = pytesseract.image_to_string(img, lang='vie')
            full_text += text + "\n"
        except Exception as e:
            print(f"\n   ⚠️ Lỗi trang {i+1}: {str(e)[:50]}")
            continue
    
    print(f"\n   ✓ Hoàn thành OCR: {total} trang")
    return full_text


In [12]:
def normalize_text(text):
    """Chuẩn hóa văn bản: loại bỏ khoảng trắng thừa, xuống dòng thừa"""
    text = text.replace("\r", "")
    text = text.replace("  ", " ")
    text = text.replace("\n\n", "\n")
    return text.strip()


In [13]:
def parse_legal_document(text):
    """
    Phân tích văn bản pháp luật, giữ nguyên cấu trúc Điều, Khoản, Điểm
    Không diễn giải, không rút gọn
    """
    # Khởi tạo cấu trúc
    document = {
        "dieu": []
    }
    
    # Tách các dòng
    lines = text.split('\n')
    current_dieu = None
    current_khoan = None
    current_diem = None
    buffer = []
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Nhận diện Điều (Điều X, Điều X., ĐIỀU X, etc.)
        dieu_match = re.match(r'^(Điều|ĐIỀU)\s+(\d+[\w]*)[\.:]?\s*(.*)', line, re.IGNORECASE)
        if dieu_match:
            # Lưu dữ liệu cũ
            if current_diem and buffer:
                current_diem["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            elif current_khoan and buffer:
                current_khoan["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            elif current_dieu and buffer:
                current_dieu["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            
            # Tạo Điều mới
            current_dieu = {
                "so_dieu": dieu_match.group(2),
                "tieu_de": dieu_match.group(3).strip() if dieu_match.group(3) else "",
                "noi_dung": "",
                "khoan": []
            }
            document["dieu"].append(current_dieu)
            current_khoan = None
            current_diem = None
            continue
        
        # Nhận diện Khoản (1. , 2. , etc. ở đầu dòng)
        khoan_match = re.match(r'^(\d+)\.\s+(.*)', line)
        if khoan_match and current_dieu:
            # Lưu dữ liệu cũ
            if current_diem and buffer:
                current_diem["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            elif current_khoan and buffer:
                current_khoan["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            
            # Tạo Khoản mới
            current_khoan = {
                "so_khoan": khoan_match.group(1),
                "noi_dung": khoan_match.group(2).strip(),
                "diem": []
            }
            current_dieu["khoan"].append(current_khoan)
            current_diem = None
            continue
        
        # Nhận diện Điểm (a), b), c), etc.)
        diem_match = re.match(r'^([a-z]|[đ])\)\s+(.*)', line, re.IGNORECASE)
        if diem_match and current_khoan:
            # Lưu dữ liệu cũ
            if current_diem and buffer:
                current_diem["noi_dung"] = ' '.join(buffer).strip()
                buffer = []
            
            # Tạo Điểm mới
            current_diem = {
                "so_diem": diem_match.group(1),
                "noi_dung": diem_match.group(2).strip()
            }
            current_khoan["diem"].append(current_diem)
            continue
        
        # Nội dung tiếp theo (thêm vào buffer)
        buffer.append(line)
    
    # Lưu phần cuối
    if current_diem and buffer:
        current_diem["noi_dung"] += ' ' + ' '.join(buffer).strip()
    elif current_khoan and buffer:
        current_khoan["noi_dung"] += ' ' + ' '.join(buffer).strip()
    elif current_dieu and buffer:
        current_dieu["noi_dung"] += ' ' + ' '.join(buffer).strip()
    
    return document


In [14]:
# # WORKFLOW CHÍNH: Xử lý PDF pháp luật
# import os

# print("=== HỆ THỐNG TRÍCH XUẤT VĂN BẢN PHÁP LUẬT ===\n")

# # Bước 1: Sử dụng pdfplumber để phân loại PDF
# print("Đang phân tích loại PDF...")
# is_text = is_text_pdf(pdf_path)
# print(f"Kết quả: {'PDF dạng TEXT (có thể trích xuất trực tiếp)' if is_text else 'PDF dạng SCAN (cần OCR)'}\n")

# # Bước 2: Trích xuất văn bản theo loại PDF
# if is_text:
#     print("Đang trích xuất text từ PDF bằng pdfplumber...")
#     raw_text = extract_text_pdf(pdf_path)
# else:
#     print("Đang chuyển đổi PDF sang ảnh...")
#     images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
#     print(f"✓ Đã chuyển đổi {len(images)} trang\n")
    
#     print("Đang thực hiện OCR với Tesseract...")
#     raw_text = ocr_from_pdf_images(images)

# print(f"✓ Đã trích xuất {len(raw_text)} ký tự\n")

# # Bước 3: Chuẩn hóa văn bản
# print("Đang chuẩn hóa văn bản...")
# raw_text = normalize_text(raw_text)
# print(f"✓ Sau chuẩn hóa: {len(raw_text)} ký tự\n")

# # Bước 4: Phân tích cấu trúc văn bản pháp luật
# print("Đang phân tích cấu trúc Điều, Khoản, Điểm...")
# parsed_data = parse_legal_document(raw_text)

# # Bước 5: Chuyển đổi sang JSON
# json_output = json.dumps(parsed_data, ensure_ascii=False, indent=2)

# # Bước 6: Lưu kết quả
# output_path = os.path.splitext(pdf_path)[0] + "_extracted.json"
# with open(output_path, 'w', encoding='utf-8') as f:
#     f.write(json_output)

# print(f"\n{'='*60}")
# print(f"✓ HOÀN THÀNH!")
# print(f"{'='*60}")
# print(f"📄 File JSON: {output_path}")
# print(f"📊 Số Điều: {len(parsed_data['dieu'])}")
# print(f"📏 Kích thước JSON: {len(json_output):,} ký tự")
# print(f"{'='*60}\n")

# # Hiển thị preview kết quả
# print("=== PREVIEW KẾT QUẢ JSON (2000 ký tự đầu) ===")
# print(json_output[:2000] + "..." if len(json_output) > 2000 else json_output)


In [15]:
# # Phân tích cấu trúc file PDF mẫu
# import pdfplumber

# sample_pdf = r"d:\NCKH\DataPhuc\120-cp.signed_qlyBTP.pdf"

# with pdfplumber.open(sample_pdf) as pdf:
#     print(f"Tổng số trang: {len(pdf.pages)}\n")
#     print("="*70)
#     print("NỘI DUNG 3 TRANG ĐẦU TIÊN:")
#     print("="*70)
    
#     for i in range(min(3, len(pdf.pages))):
#         page = pdf.pages[i]
#         text = page.extract_text()
#         print(f"\n{'='*70}")
#         print(f"TRANG {i+1}")
#         print('='*70)
#         print(text[:2000] if text and len(text) > 2000 else text or "[Không trích xuất được]")
#         if text and len(text) > 2000:
#             print("\n[...còn tiếp...]")


## 📋 CẤU TRÚC JSON HỢP LÝ CHO VĂN BẢN PHÁP LUẬT VIỆT NAM

### Phân tích yêu cầu:
1. **Giữ nguyên cấu trúc**: Điều, Khoản, Điểm
2. **Không diễn giải, không rút gọn**: Lưu toàn bộ nội dung gốc
3. **Metadata đầy đủ**: Thông tin văn bản, cơ quan ban hành, ngày tháng

### Cấu trúc JSON được đề xuất:

```json
{
  "metadata": {
    "loai_van_ban": "Nghị định",
    "so_hieu": "120/2025/NĐ-CP",
    "trich_yeu": "Quy định về...",
    "co_quan_ban_hanh": "Chính phủ",
    "nguoi_ky": "Thủ tướng Chính phủ",
    "ngay_ban_hanh": "DD/MM/YYYY",
    "ngay_hieu_luc": "DD/MM/YYYY",
    "so_trang": 14,
    "can_cu_phap_ly": ["Luật...", "Nghị quyết..."]
  },
  "phan_mo_dau": {
    "co_quan_ban_hanh": "...",
    "ten_quoc_gia": "...",
    "quoc_hieu": "...",
    "so_hieu": "...",
    "noi_ban_hanh": "...",
    "ngay_thang_nam": "..."
  },
  "ten_van_ban": "NGHỊ ĐỊNH...",
  "can_cu_phap_ly": [
    "Căn cứ Luật...",
    "Căn cứ Nghị quyết..."
  ],
  "theo_de_nghi": "Theo đề nghị của Bộ trưởng Bộ...",
  "phan_quyet_dinh": "Chính phủ ban hành Nghị định...",
  "chuong": [
    {
      "so_chuong": "I",
      "ten_chuong": "QUY ĐỊNH CHUNG",
      "dieu": [
        {
          "so_dieu": "1",
          "tieu_de": "Phạm vi điều chỉnh",
          "noi_dung_tong_quat": "Nội dung không thuộc khoản nào",
          "khoan": [
            {
              "so_khoan": "1",
              "noi_dung": "Nội dung khoản 1...",
              "diem": [
                {
                  "so_diem": "a",
                  "noi_dung": "Nội dung điểm a..."
                },
                {
                  "so_diem": "b",
                  "noi_dung": "Nội dung điểm b..."
                }
              ]
            },
            {
              "so_khoan": "2",
              "noi_dung": "Nội dung khoản 2...",
              "diem": []
            }
          ]
        }
      ]
    }
  ],
  "dieu_khoan_thi_hanh": {
    "hieu_luc": "Nghị định này có hiệu lực...",
    "trach_nhiem_thi_hanh": "Các Bộ trưởng, Thủ trưởng...",
    "noi_nhan": ["...", "..."]
  }
}
```

### Ưu điểm của cấu trúc này:

✅ **Đầy đủ metadata**: Số hiệu, ngày ban hành, cơ quan...  
✅ **Phân cấp rõ ràng**: Chương → Điều → Khoản → Điểm  
✅ **Giữ nguyên nội dung**: Không diễn giải hay rút gọn  
✅ **Dễ truy vấn**: Có thể tìm kiếm theo Điều, Khoản, Điểm  
✅ **Mở rộng được**: Có thể thêm trường mới nếu cần


In [19]:
def parse_legal_document_v2(text):
    """
    Phân tích văn bản pháp luật với cấu trúc JSON đầy đủ
    Bao gồm: Metadata, Chương, Điều, Khoản, Điểm
    """
    import re
    
    document = {
        "metadata": {
            "loai_van_ban": "",
            "so_hieu": "",
            "trich_yeu": "",
            "co_quan_ban_hanh": "",
            "nguoi_ky": "",
            "ngay_ban_hanh": "",
            "ngay_hieu_luc": "",
            "so_trang": 0
        },
        "phan_mo_dau": {
            "co_quan_ban_hanh": "",
            "ten_quoc_gia": "",
            "quoc_hieu": "",
            "so_hieu": "",
            "noi_ban_hanh": "",
            "ngay_thang_nam": ""
        },
        "ten_van_ban": "",
        "can_cu_phap_ly": [],
        "theo_de_nghi": "",
        "phan_quyet_dinh": "",
        "chuong": []
    }
    
    lines = text.split('\n')
    
    # Trích xuất metadata từ đầu văn bản
    for i, line in enumerate(lines[:50]):  # Chỉ xem 50 dòng đầu
        line = line.strip()
        
        # Tìm số hiệu
        if re.search(r'Số:\s*(\d+[/\d\w\-]+)', line, re.IGNORECASE):
            match = re.search(r'Số:\s*(\d+[/\d\w\-]+)', line, re.IGNORECASE)
            document["metadata"]["so_hieu"] = match.group(1)
            document["phan_mo_dau"]["so_hieu"] = match.group(1)
        
        # Tìm loại văn bản
        if re.search(r'(NGHỊ ĐỊNH|QUYẾT ĐỊNH|THÔNG TƯ|LUẬT|PHÁP LỆNH)', line, re.IGNORECASE):
            match = re.search(r'(NGHỊ ĐỊNH|QUYẾT ĐỊNH|THÔNG TƯ|LUẬT|PHÁP LỆNH)', line, re.IGNORECASE)
            document["metadata"]["loai_van_ban"] = match.group(1)
        
        # Tìm ngày ban hành
        if re.search(r'ngày\s+(\d{1,2})\s+tháng\s+(\d{1,2})\s+năm\s+(\d{4})', line, re.IGNORECASE):
            match = re.search(r'ngày\s+(\d{1,2})\s+tháng\s+(\d{1,2})\s+năm\s+(\d{4})', line, re.IGNORECASE)
            document["metadata"]["ngay_ban_hanh"] = f"{match.group(1)}/{match.group(2)}/{match.group(3)}"
            document["phan_mo_dau"]["ngay_thang_nam"] = line
        
        # Tìm cơ quan ban hành
        if "Cơ quan phát hành:" in line or "CHÍNH PHỦ" in line:
            document["metadata"]["co_quan_ban_hanh"] = line.replace("Cơ quan phát hành:", "").strip()
            document["phan_mo_dau"]["co_quan_ban_hanh"] = line
    
    # Phân tích cấu trúc Chương - Điều - Khoản - Điểm
    current_chuong = None
    current_dieu = None
    current_khoan = None
    current_diem = None
    buffer = []
    can_cu_buffer = []
    is_can_cu = False
    is_collecting_chuong_name = False
    chuong_name_buffer = []
    is_collecting_dieu_title = False
    dieu_title_buffer = []
    is_collecting_khoan_content = False
    khoan_content_buffer = []
    is_collecting_diem_content = False
    diem_content_buffer = []
    
    for line in lines:
        line_stripped = line.strip()
        if not line_stripped:
            continue
        
        # Nhận diện Căn cứ pháp lý
        if re.match(r'^Căn cứ\s+', line_stripped, re.IGNORECASE):
            is_can_cu = True
            can_cu_buffer.append(line_stripped)
            continue
        elif is_can_cu and line_stripped.endswith(';'):
            can_cu_buffer.append(line_stripped)
            document["can_cu_phap_ly"].append(' '.join(can_cu_buffer))
            can_cu_buffer = []
            is_can_cu = False
            continue
        elif is_can_cu:
            can_cu_buffer.append(line_stripped)
            continue
        
        # Nhận diện Chương
        chuong_match = re.match(r'^(Chương|CHƯƠNG)\s+([IVXLCDM]+|[\dA-Z]+)[\.:]?\s*(.*)', line_stripped, re.IGNORECASE)
        if chuong_match:
            current_chuong = {
                "so_chuong": chuong_match.group(2),
                "ten_chuong": "",  # Sẽ được cập nhật sau
                "dieu": []
            }
            document["chuong"].append(current_chuong)
            current_dieu = None
            current_khoan = None
            current_diem = None
            buffer = []
            # Bắt đầu thu thập tên chương (có thể nhiều dòng)
            is_collecting_chuong_name = True
            chuong_name_buffer = []
            if chuong_match.group(3).strip():
                chuong_name_buffer.append(chuong_match.group(3).strip())
            continue
        
        # Nhận diện Điều CHÍNH (phải có dấu chấm hoặc hai chấm SAU số, không phải tham chiếu)
        # Ví dụ: "Điều 4." hoặc "Điều 4:" → Điều chính
        # Loại trừ: "Điều 4 Nghị quyết" → tham chiếu
        dieu_match = re.match(r'^(Điều|ĐIỀU)\s+(\d+[\w]*)[\.:]\s*(.*)', line_stripped, re.IGNORECASE)
        if dieu_match:
            # Kiểm tra có phải tham chiếu văn bản khác không
            rest_text = dieu_match.group(3).strip()
            # Nếu ngay sau "Điều X." có tên văn bản → đây là tham chiếu, không phải Điều chính
            if re.match(r'^(Nghị quyết|Nghị định|Quyết định|Thông tư|Luật|Pháp lệnh|NĐ-CP|QH\d+)', rest_text, re.IGNORECASE):
                # Đây là tham chiếu, không tạo Điều mới, thêm vào nội dung hiện tại
                if is_collecting_dieu_title:
                    dieu_title_buffer.append(line_stripped)
                elif is_collecting_diem_content:
                    diem_content_buffer.append(line_stripped)
                elif is_collecting_khoan_content:
                    khoan_content_buffer.append(line_stripped)
                else:
                    buffer.append(line_stripped)
                continue
            
            # Đây là Điều chính, tạo Điều mới
            # Hoàn thiện tên chương nếu đang thu thập
            if is_collecting_chuong_name and current_chuong:
                current_chuong["ten_chuong"] = ' '.join(chuong_name_buffer).strip()
                is_collecting_chuong_name = False
                chuong_name_buffer = []
            
            # Hoàn thiện tiêu đề Điều cũ
            if is_collecting_dieu_title and current_dieu:
                current_dieu["tieu_de"] = ' '.join(dieu_title_buffer).strip()
                is_collecting_dieu_title = False
                dieu_title_buffer = []
            
            # Hoàn thiện nội dung Điểm cũ
            if is_collecting_diem_content and current_diem:
                current_diem["noi_dung"] = ' '.join(diem_content_buffer).strip()
                is_collecting_diem_content = False
                diem_content_buffer = []
            # Hoàn thiện nội dung Khoản cũ
            elif is_collecting_khoan_content and current_khoan:
                current_khoan["noi_dung"] = ' '.join(khoan_content_buffer).strip()
                is_collecting_khoan_content = False
                khoan_content_buffer = []
            
            # Tạo Điều mới
            current_dieu = {
                "so_dieu": dieu_match.group(2),
                "tieu_de": "",
                "khoan": []
            }
            
            # Thêm vào Chương hoặc root
            if current_chuong:
                current_chuong["dieu"].append(current_dieu)
            else:
                # Nếu chưa có chương, tạo chương mặc định
                if not document["chuong"]:
                    document["chuong"].append({
                        "so_chuong": "",
                        "ten_chuong": "",
                        "dieu": []
                    })
                    current_chuong = document["chuong"][0]
                current_chuong["dieu"].append(current_dieu)
            
            current_khoan = None
            current_diem = None
            # Bắt đầu thu thập tiêu đề Điều (nhiều dòng chữ in đậm)
            is_collecting_dieu_title = True
            dieu_title_buffer = []
            # Thêm phần text cùng dòng với số Điều (nếu có)
            if dieu_match.group(3).strip():
                dieu_title_buffer.append(dieu_match.group(3).strip())
            continue
        
        # Nhận diện Khoản (CHỈ khi có cả số VÀ nội dung)
        khoan_match = re.match(r'^(\d+)\.\s+(.+)', line_stripped)
        if khoan_match and current_dieu and len(khoan_match.group(2).strip()) > 0:
            # Hoàn thiện tiêu đề Điều nếu đang thu thập
            if is_collecting_dieu_title:
                current_dieu["tieu_de"] = ' '.join(dieu_title_buffer).strip()
                is_collecting_dieu_title = False
                dieu_title_buffer = []
            
            # Hoàn thiện nội dung Điểm cũ
            if is_collecting_diem_content and current_diem:
                current_diem["noi_dung"] = ' '.join(diem_content_buffer).strip()
                is_collecting_diem_content = False
                diem_content_buffer = []
            # Hoàn thiện nội dung Khoản cũ
            elif is_collecting_khoan_content and current_khoan:
                current_khoan["noi_dung"] = ' '.join(khoan_content_buffer).strip()
                is_collecting_khoan_content = False
                khoan_content_buffer = []
            
            # Tạo Khoản mới
            current_khoan = {
                "so_khoan": khoan_match.group(1),
                "noi_dung": khoan_match.group(2).strip(),
                "diem": []
            }
            current_dieu["khoan"].append(current_khoan)
            current_diem = None
            # Bắt đầu thu thập nội dung Khoản
            is_collecting_khoan_content = True
            khoan_content_buffer = [khoan_match.group(2).strip()]
            continue
        
        # Nhận diện Điểm (bao gồm cả "1)" - OCR nhầm "l)" thành "1)")
        diem_match = re.match(r'^([a-z]|đ|1)\)\s+(.*)', line_stripped, re.IGNORECASE)
        if diem_match and current_khoan:
            # Nếu match "1)" nhưng chưa có điểm nào → đây là Khoản 1, không phải Điểm
            # Nếu match "1)" và đã có điểm → đây là Điểm "l" bị OCR nhầm
            if diem_match.group(1) == "1" and len(current_khoan["diem"]) == 0:
                # Đây là Khoản 1, không phải Điểm, bỏ qua và xử lý như Khoản bên dưới
                pass
            else:
                # Đây là Điểm (hoặc "l" bị nhầm thành "1")
                # Hoàn thiện nội dung Điểm cũ
                if is_collecting_diem_content and current_diem:
                    current_diem["noi_dung"] = ' '.join(diem_content_buffer).strip()
                    is_collecting_diem_content = False
                    diem_content_buffer = []
                # Dừng thu thập nội dung Khoản
                elif is_collecting_khoan_content:
                    is_collecting_khoan_content = False
                    if khoan_content_buffer:
                        current_khoan["noi_dung"] = ' '.join(khoan_content_buffer).strip()
                        khoan_content_buffer = []
                
                # Tạo Điểm mới (nếu "1" thì đổi thành "l")
                so_diem = diem_match.group(1)
                if so_diem == "1":
                    so_diem = "l"  # OCR nhầm, sửa lại
                
                current_diem = {
                    "so_diem": so_diem,
                    "noi_dung": diem_match.group(2).strip()
                }
                current_khoan["diem"].append(current_diem)
                # Bắt đầu thu thập nội dung Điểm
                is_collecting_diem_content = True
                diem_content_buffer = [diem_match.group(2).strip()]
                continue
        
        # Nhận diện Khoản (CHỈ khi có cả số VÀ nội dung)
        # Đặt sau Điểm để ưu tiên Điểm trước
        khoan_match = re.match(r'^(\d+)\.\s+(.+)', line_stripped)
        if khoan_match and current_dieu and len(khoan_match.group(2).strip()) > 0:
            # Hoàn thiện tiêu đề Điều nếu đang thu thập
            if is_collecting_dieu_title:
                current_dieu["tieu_de"] = ' '.join(dieu_title_buffer).strip()
                is_collecting_dieu_title = False
                dieu_title_buffer = []
            
            # Hoàn thiện nội dung Điểm cũ
            if is_collecting_diem_content and current_diem:
                current_diem["noi_dung"] = ' '.join(diem_content_buffer).strip()
                is_collecting_diem_content = False
                diem_content_buffer = []
            # Hoàn thiện nội dung Khoản cũ
            elif is_collecting_khoan_content and current_khoan:
                current_khoan["noi_dung"] = ' '.join(khoan_content_buffer).strip()
                is_collecting_khoan_content = False
                khoan_content_buffer = []
            
            # Tạo Khoản mới
            current_khoan = {
                "so_khoan": khoan_match.group(1),
                "noi_dung": khoan_match.group(2).strip(),
                "diem": []
            }
            current_dieu["khoan"].append(current_khoan)
            current_diem = None
            # Bắt đầu thu thập nội dung Khoản
            is_collecting_khoan_content = True
            khoan_content_buffer = [khoan_match.group(2).strip()]
            continue
        
        # Thu thập tên chương nhiều dòng
        if is_collecting_chuong_name:
            chuong_name_buffer.append(line_stripped)
            continue
        
        # Thu thập tiêu đề Điều nhiều dòng (chữ in đậm)
        if is_collecting_dieu_title:
            dieu_title_buffer.append(line_stripped)
            continue
        
        # Thu thập nội dung Điểm nhiều dòng
        if is_collecting_diem_content:
            diem_content_buffer.append(line_stripped)
            continue
        
        # Thu thập nội dung Khoản nhiều dòng
        if is_collecting_khoan_content:
            khoan_content_buffer.append(line_stripped)
            continue
        
        # Nội dung tiếp theo
        buffer.append(line_stripped)
    
    # Lưu buffer cuối
    if is_collecting_dieu_title and current_dieu:
        current_dieu["tieu_de"] = ' '.join(dieu_title_buffer).strip()
    elif is_collecting_diem_content and current_diem:
        current_diem["noi_dung"] = ' '.join(diem_content_buffer).strip()
    elif is_collecting_khoan_content and current_khoan:
        current_khoan["noi_dung"] = ' '.join(khoan_content_buffer).strip()
    elif current_diem and buffer:
        current_diem["noi_dung"] += ' ' + ' '.join(buffer).strip()
    elif current_khoan and buffer:
        current_khoan["noi_dung"] += ' ' + ' '.join(buffer).strip()
    
    return document


## 🚀 SỬ DỤNG PHIÊN BẢN MỚI

Chạy cell bên dưới để xử lý PDF với cấu trúc JSON đầy đủ hơn

In [20]:
# WORKFLOW V2: Xử lý với cấu trúc JSON đầy đủ
import os
from tkinter import Tk, filedialog

print("="*70)
print("HỆ THỐNG TRÍCH XUẤT VĂN BẢN PHÁP LUẬT - PHIÊN BẢN 2")
print("="*70)
print()


# Bước 1: Phân loại PDF
print("📋 BƯỚC 1: Phân tích loại PDF...")
is_text = is_text_pdf(pdf_path)
print(f"   Kết quả: {'✓ PDF Text (trích xuất trực tiếp)' if is_text else '✓ PDF Scan (cần OCR)'}")
print()

# Bước 2: Trích xuất văn bản
print("📄 BƯỚC 2: Trích xuất nội dung...")
if is_text:
    raw_text = extract_text_pdf(pdf_path)
else:
    images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
    print(f"   ✓ Đã chuyển {len(images)} trang sang ảnh")
    raw_text = ocr_from_pdf_images(images)

print(f"   ✓ Trích xuất: {len(raw_text):,} ký tự")
print()

# Bước 3: Chuẩn hóa
print("🔧 BƯỚC 3: Chuẩn hóa văn bản...")
raw_text = normalize_text(raw_text)
print(f"   ✓ Sau chuẩn hóa: {len(raw_text):,} ký tự")
print()

# Bước 4: Parse với cấu trúc mới
print("⚙️  BƯỚC 4: Phân tích cấu trúc...")
parsed_v2 = parse_legal_document_v2(raw_text)
print(f"   ✓ Số hiệu: {parsed_v2['metadata']['so_hieu'] or 'Chưa xác định'}")
print(f"   ✓ Loại văn bản: {parsed_v2['metadata']['loai_van_ban'] or 'Chưa xác định'}")
print(f"   ✓ Số Chương: {len(parsed_v2['chuong'])}")

total_dieu = sum(len(ch['dieu']) for ch in parsed_v2['chuong'])
print(f"   ✓ Tổng số Điều: {total_dieu}")
print()

# Bước 5: Xuất JSON
print("💾 BƯỚC 5: Lưu file JSON...")
json_v2 = json.dumps(parsed_v2, ensure_ascii=False, indent=2)

# Cho phép chọn nơi lưu file
default_filename = os.path.splitext(os.path.basename(pdf_path))[0] + "_v2.json"
output_v2 = filedialog.asksaveasfilename(
    defaultextension=".json",
    filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
    initialfile=default_filename,
    title="Chọn nơi lưu file JSON"
)

if not output_v2:
    print("   ⚠️ Không chọn file, hủy lưu.")
else:
    with open(output_v2, 'w', encoding='utf-8') as f:
        f.write(json_v2)

    print(f"   ✓ File JSON: {output_v2}")
    print(f"   ✓ Kích thước: {len(json_v2):,} ký tự")
    print()

    print("="*70)
    print("✅ HOÀN THÀNH!")
    print("="*70)
    print()

    # Preview
    print("📊 PREVIEW KẾT QUẢ (metadata + cấu trúc đầu tiên):")
    print("-"*70)
    print(json_v2[:3000] + "\n..." if len(json_v2) > 3000 else json_v2)

HỆ THỐNG TRÍCH XUẤT VĂN BẢN PHÁP LUẬT - PHIÊN BẢN 2

📋 BƯỚC 1: Phân tích loại PDF...
   Kết quả: ✓ PDF Scan (cần OCR)

📄 BƯỚC 2: Trích xuất nội dung...
   ✓ Đã chuyển 108 trang sang ảnh
   🔄 Đang xử lý: 108/108 trang...
   ✓ Hoàn thành OCR: 108 trang
   ✓ Trích xuất: 187,486 ký tự

🔧 BƯỚC 3: Chuẩn hóa văn bản...
   ✓ Sau chuẩn hóa: 185,988 ký tự

⚙️  BƯỚC 4: Phân tích cấu trúc...
   ✓ Số hiệu: 144/2025/NĐ-CP
   ✓ Loại văn bản: Luật
   ✓ Số Chương: 2
   ✓ Tổng số Điều: 58

💾 BƯỚC 5: Lưu file JSON...
   ✓ File JSON: D:/NCKH/DataPhuc/pdf_to_json/144-ndcp.signed_bộ xây dựng_v2.json
   ✓ Kích thước: 255,331 ký tự

✅ HOÀN THÀNH!

📊 PREVIEW KẾT QUẢ (metadata + cấu trúc đầu tiên):
----------------------------------------------------------------------
{
  "metadata": {
    "loai_van_ban": "Luật",
    "so_hieu": "144/2025/NĐ-CP",
    "trich_yeu": "",
    "co_quan_ban_hanh": "sản tộc TIM ĐIỆN TỪ CHÍNH PHỦ NGHỊ ĐỊNH",
    "nguoi_ky": "",
    "ngay_ban_hanh": "19/02/2025",
    "ngay_hieu_luc": "",
